In [ ]:
# Disturbingly, this represents 2.5 hours work!

In [3]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import gc
import re

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [4]:
%%time
# Load BES Panel for Wave 8
raw_data = pd.read_csv("The Man of 2017 (Responses) - Form Responses 1.csv")
#(31409, 733)

# BES_Panel = pd.read_stata("../BES_analysis_data/BES2015_W8_v1.6.dta")

print( raw_data.shape )

(313, 73)
Wall time: 13 ms


In [5]:
raw_data.describe()

,Timestamp,How would you describe your gender?,Which of the following would you describe yourself as?,Where do you live?,What is your age?,Do you believe that a secret cabal of lizard-people rule the world?,"A man has the confidence that if something is broken, it can be fixed.",A man cultivates as many interests as is possible.,"A man knows everything about something, and something about everything.",A man appreciates beauty in all of its forms.,...,A man does not let his feelings cloud his judgement.,"A man is secure in his convictions, and is not easily swayed by the words of others.",A man is ready to use physical force or violence where it is necessary.,A man cherishes and values the qualities of women.,A man will defend his integrity if it is questioned.,A man helps other men to be good men.,A man does not speak ill of others.,A man is faithful to his roots.,A man lives in the present.,Feedback
count,313,313,309,313,313,313,313,313,313,313,...,313,313,313,313,313,313,313,313,313,54
unique,313,3,80,6,4,3,4,4,4,4,...,4,4,4,4,4,4,4,4,4,54
top,12/13/2016 22:23:21,Man,"Socialist, Liberal, Feminist",Europe,26 - 40,No,Desirable,Desirable,Neutral,Desirable,...,Neutral,Neutral,Neutral,Essential,Desirable,Desirable,Desirable,Neutral,Neutral,"For pretty much every answer, substitute ""men""..."
freq,1,189,35,283,205,259,153,140,166,190,...,131,182,155,145,155,142,150,216,188,1


In [6]:
parsed_data = raw_data.copy()


In [7]:
# datetime
parsed_data["Timestamp"] = raw_data["Timestamp"].apply(lambda x: pd.to_datetime(x) )

In [8]:
# Not Ordinal
parsed_data["How would you describe your gender?"] = raw_data["How would you describe your gender?"].astype('category')
parsed_data["Where do you live?"] = raw_data["Where do you live?"].astype('category')

In [9]:
# Ordinal
parsed_data["What is your age?"] = \
   raw_data["What is your age?"].astype('category')\
   .cat.set_categories(['Under 18','18 - 25', '26 - 40', '41 - 65' ], ordered=True)

parsed_data["Do you believe that a secret cabal of lizard-people rule the world?"] = \
   raw_data["Do you believe that a secret cabal of lizard-people rule the world?"] \
   .astype('category').cat.set_categories(["No","Maybe","Yes"], ordered=True)

In [10]:
# Text/Vectorisable
# Feedback object (text)
   # Could derive boolean Feedback_given
# Which of the following would you describe yourself as? object(text)
   # Could derive set of booleans for each subcategory

In [11]:
special_case_columns = ["Timestamp",
                        "How would you describe your gender?", "Where do you live?",
                        "What is your age?", "Do you believe that a secret cabal of lizard-people rule the world?",
                        "Feedback", "Which of the following would you describe yourself as?" ]

undes_essential_columns = [x for x in raw_data.columns if x not in special_case_columns]

for col in undes_essential_columns:
    parsed_data[col] = raw_data[col]\
        .astype('category').cat.set_categories(["Undesirable", "Neutral", "Desirable", "Essential"], ordered=True)

In [12]:
multi_cat_col = "Which of the following would you describe yourself as?"
category_list = []
for ind in raw_data[ raw_data[multi_cat_col].notnull() ].index:
    described_self_as = raw_data[multi_cat_col][ind]
    sub_categories = described_self_as.split(",")
    for cat in sub_categories:
        cat_stripped = cat.strip()
        if cat_stripped not in category_list:
            category_list.append( cat_stripped )
            parsed_data[cat_stripped] = False # initialise
        parsed_data.loc[ind,cat_stripped]=True
            
category_list

['Socialist',
 'Liberal',
 'Feminist',
 'Egalitarian',
 'Libertarian',
 'Anarchist',
 'Moderate',
 'Conservative',
 "Men's Rights Activist"]

In [13]:
parsed_data["Feedback_length"] = raw_data["Feedback"].apply(lambda x: len(x) if not isinstance(x,float ) else 0 )

In [14]:
parsed_data.dtypes

Timestamp                                                                                                    datetime64[ns]
How would you describe your gender?                                                                                category
Which of the following would you describe yourself as?                                                               object
Where do you live?                                                                                                 category
What is your age?                                                                                                  category
Do you believe that a secret cabal of lizard-people rule the world?                                                category
A man has the confidence that if something is broken, it can be fixed.                                             category
A man cultivates as many interests as is possible.                                                                 category
A man kn

In [15]:
new_col_list = []
ind = 0
column_name_key = pd.DataFrame(columns = ["new_column_name", "original_column_name"])
for col in parsed_data.columns:
    match_str = re.match("A man[^\s]* (.*)",col)
    if match_str:
        subbed_str = re.sub("[^a-zA-Z0-9_]+","_",  match_str.groups()[0]).strip("_")
    else:
        subbed_str = re.sub("[^a-zA-Z0-9_]+","_",  col ).strip("_")
        
    word_array = np.array( [len(x) for x in subbed_str.split("_")] )+1
    max_words = np.where( np.cumsum( word_array )<33 )[0].max()        
    new_col_str = "_".join( subbed_str.split("_")[0:max_words+1] ).lower()
    print( new_col_str )
    if len(new_col_str)>32:
        raise Exception("column name too long!", new_col_str)
    new_col_list.append(new_col_str)        
        
    column_name_key.loc[ind] = [new_col_str,col]
    ind = ind +1
    
parsed_data.columns = column_name_key["new_column_name"].values

timestamp
how_would_you_describe_your
which_of_the_following_would
where_do_you_live
what_is_your_age
do_you_believe_that_a_secret
has_the_confidence_that_if
cultivates_as_many_interests_as
knows_everything_about
appreciates_beauty_in_all_of
uses_his_wealth_and_social
does_what_he_says_he_will_do
may_take_a_beating_of_whatever
shows_courtesy_by_holding_doors
is_willing_to_endure_suffering
protects_those_who_are_less
arrives_to_occasions_on_time
always_looks_to_improve_himself
is_willing_to_challenge_the
shows_sportsmanship_and_does
speaks_his_mind
settles_disputes_by_applying
is_financially_independent
doesn_t_let_the_little
is_unashamed_of_his_sexual
is_willing_and_able_to_please
does_not_make_demands_of_his
seeks_advice_when_he_wants_to
does_not_rely_on_the_kindness
is_quick_to_respond_in_a_time
does_not_let_fear_or
powers_through_a_problem_when
cultivates_practical_skills_and
puts_the_lives_of_others_over
maintains_his_physical_strength
would_rather_try_and_fail_than
has_a_plan_to_g

In [16]:
column_name_key.to_csv("new_column_name_key.csv")
parsed_data.to_pickle("parsed_data.pkl")

In [17]:
parsed_data["timestamp"] = parsed_data["timestamp"].astype('str')
parsed_data.drop(["feedback"],axis=1) .to_stata("man_of_2017_parsed.dta")

In [18]:
parsed_data = pd.read_pickle("man_of_2017_parsed.pkl")

In [19]:
parsed_data

,timestamp,how_would_you_describe_your,which_of_the_following_would,where_do_you_live,what_is_your_age,do_you_believe_that_a_secret,has_the_confidence_that_if,cultivates_as_many_interests_as,knows_everything_about,appreciates_beauty_in_all_of,...,socialist,liberal,feminist,egalitarian,libertarian,anarchist,moderate,conservative,men_s_rights_activist,feedback_length
0,2016-11-19 09:15:49,Man,"Socialist, Liberal, Feminist",Europe,41 - 65,No,Neutral,Neutral,Neutral,Neutral,...,True,True,True,False,False,False,False,False,False,0
1,2016-11-19 09:19:33,Woman,Feminist,Europe,26 - 40,No,Desirable,Desirable,Neutral,Essential,...,False,False,True,False,False,False,False,False,False,0
2,2016-11-19 09:25:32,Woman,"Socialist, Liberal, Feminist",Europe,26 - 40,No,Desirable,Desirable,Neutral,Desirable,...,True,True,True,False,False,False,False,False,False,0
3,2016-11-19 09:27:00,Man,"Socialist, Liberal, Feminist",Europe,26 - 40,No,Desirable,Neutral,Desirable,Desirable,...,True,True,True,False,False,False,False,False,False,0
4,2016-11-19 09:31:15,Other,"Socialist, Liberal, Feminist",Europe,18 - 25,No,Desirable,Desirable,Neutral,Desirable,...,True,True,True,False,False,False,False,False,False,0
5,2016-11-19 09:34:21,Woman,Feminist,Europe,26 - 40,No,Undesirable,Essential,Undesirable,Essential,...,False,False,True,False,False,False,False,False,False,0
6,2016-11-19 09:35:34,Woman,Feminist,Europe,26 - 40,No,Desirable,Desirable,Desirable,Neutral,...,False,False,True,False,False,False,False,False,False,0
7,2016-11-19 09:38:24,Woman,"Socialist, Liberal, Feminist",Europe,26 - 40,No,Desirable,Desirable,Desirable,Desirable,...,True,True,True,False,False,False,False,False,False,113
8,2016-11-19 09:39:21,Other,"Socialist, Liberal, Feminist, Egalitarian",Europe,26 - 40,No,Desirable,Desirable,Undesirable,Desirable,...,True,True,True,True,False,False,False,False,False,0
9,2016-11-19 09:39:49,Man,"Socialist, Liberal, Libertarian, Anarchist, Fe...",Europe,26 - 40,No,Neutral,Neutral,Neutral,Neutral,...,True,True,True,False,True,True,False,False,False,0


In [20]:
parsed_data.dtypes

timestamp                          datetime64[ns]
how_would_you_describe_your              category
which_of_the_following_would               object
where_do_you_live                        category
what_is_your_age                         category
do_you_believe_that_a_secret             category
has_the_confidence_that_if               category
cultivates_as_many_interests_as          category
knows_everything_about                   category
appreciates_beauty_in_all_of             category
uses_his_wealth_and_social               category
does_what_he_says_he_will_do             category
may_take_a_beating_of_whatever           category
shows_courtesy_by_holding_doors          category
is_willing_to_endure_suffering           category
protects_those_who_are_less              category
arrives_to_occasions_on_time             category
always_looks_to_improve_himself          category
is_willing_to_challenge_the              category
shows_sportsmanship_and_does             category


In [29]:
parsed_data["how_would_you_describe_your"].value_counts()

Man      189
Woman     92
Other     32
dtype: int64